In [1]:
########################### LLIBRERIES ##################################

# Importem llibreries io
import sys
import os
import pickle


# Importem les llibreries relacionades amb Pytorch
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.optim as optim
import torch.nn as nn
import torch
import gc

# Importem llibreries per la validació
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold
from sklearn.metrics import roc_curve


# Importem llibreries pel processament d'imatges
#import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import glob
import cv2

########################### FUNCIONS ####################################

# Importem les funcions propies necessaries
from Models.datasets import Standard_Dataset
from loadAnnotatedServidor import loadAnnotated
from loadCroppedServidor import loadCropped
from reconstructionerror import *
from Models.AEmodels import *

In [2]:
import pickle

# Ruta del archivo pickle
pickle_file = 'validation_100imgsConfig3.pkl'  # Ajusta esta ruta si el archivo tiene un nombre o ubicación diferente

# Cargar los datos del archivo pickle
with open(pickle_file, 'rb') as file:  # 'rb' es para lectura en modo binario
    loaded_lists = pickle.load(file)

# Mostrar el contenido del diccionario cargado (opcional)
print("Datos cargados correctamente")
#print(loaded_lists)


Datos cargados correctamente


In [3]:
OTPC = loaded_lists["OTPC"] #OptimalThresholdsPatchClassification
OTPD = loaded_lists["OTPD"] #OptimalThresholdsPatientsDiagnosis
RTPC = loaded_lists["RTPC"] #ResultsTestPatchClassification
RTPD = loaded_lists["RTPD"] #ResultsTestPatientsDiagnosis


In [4]:
import glob
import os
import numpy as np
import random
import cv2
import pandas as pd
import skimage.io as io

def loadHoldOut(ListFolders, nImgs):
    pathExcel = "PatientDiagnosis.csv"
    patientsImgs = list()
    patientsMeta= []
    excel = pd.read_csv(pathExcel)
    nImages = nImgs
    patientsRealDiagnosis = {}

    for _, row in excel.iterrows():
        cod = row['CODI']  # Columna CODI
        diagnosis = row['DENSITAT']  # Columna DENSITAT
        # Mapear los valores de diagnóstico
        if diagnosis == "NEGATIVA":
            diagnosis = -1
        elif diagnosis in ["BAIXA", "ALTA"]:
            diagnosis = 1
        patientsRealDiagnosis[cod] = diagnosis

    for pathDir in ListFolders:
        print("Loading images from", pathDir)
        pathdir_llarg = os.path.join("HoldOUt/", pathDir)
        imgs = os.listdir(pathdir_llarg)
        n = len(imgs)
        if nImgs == None:
            nImages = n
            
        this_patient = 0	
        while this_patient < min(nImages,n):
            print(len(imgs))
            if len(imgs) == 0:
                break
            index = random.randint(0,len(imgs)-1)
            imgname = imgs.pop(index)
            if imgname[-3:] != "png":
                continue
            
            imgpath = os.path.join(pathdir_llarg,imgname)
            
           
            
            img = cv2.imread(imgpath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) if img.shape[-1] >= 3 else img[:, :, :3]

             # Obtener el diagnóstico del paciente
            pacient = pathDir[:-2]
            pacient = pacient.replace("_1", "").replace("_0", "")
            print(f"Patient ID: {pacient}")
            
            # Obtener el diagnóstico real del paciente
            if pacient in patientsRealDiagnosis:
                diagnosis = patientsRealDiagnosis[pacient]
            else:
                print(f"Diagnosis not found for patient {pacient}")
                continue  # Si no encontramos el diagnóstico, continuamos con el siguiente paciente
            print("imatge carregada")
            if img.shape != (256, 256, 3):
                imgs.pop(index)
                if len(imgs) == 0:
                    break
                print("Image with wrong size found", img.size)
                continue
            
            patientsMeta.append(pacient)
            
	 		# transpose img
            img = np.transpose(img, (2, 0, 1))
			#img = img.astype(np.float32)
            img = img.astype(np.float32)
            img = img/255.0
            patientsImgs.append(img)
            this_patient += 1

    return patientsImgs, patientsMeta, patientsRealDiagnosis

In [5]:
pathDir = 'HoldOut/'
holdoutImgs, holdoutMeta, patientsRealDiagnosis = loadHoldOut(os.listdir(pathDir),100)

Loading images from B22-01_1
487
Patient ID: B22-01
imatge carregada
486
Patient ID: B22-01
imatge carregada
485
Patient ID: B22-01
imatge carregada
484
Patient ID: B22-01
imatge carregada
483
Patient ID: B22-01
imatge carregada
482
Patient ID: B22-01
imatge carregada
481
Patient ID: B22-01
imatge carregada
480
Patient ID: B22-01
imatge carregada
479
Patient ID: B22-01
imatge carregada
478
Patient ID: B22-01
imatge carregada
477
Patient ID: B22-01
imatge carregada
476
Patient ID: B22-01
imatge carregada
475
Patient ID: B22-01
imatge carregada
474
Patient ID: B22-01
imatge carregada
473
Patient ID: B22-01
imatge carregada
472
Patient ID: B22-01
imatge carregada
471
Patient ID: B22-01
imatge carregada
470
Patient ID: B22-01
imatge carregada
469
Patient ID: B22-01
imatge carregada
468
Patient ID: B22-01
imatge carregada
467
Patient ID: B22-01
imatge carregada
466
Patient ID: B22-01
imatge carregada
465
Patient ID: B22-01
imatge carregada
464
Patient ID: B22-01
imatge carregada
463
Patient

In [6]:

net_paramsEnc = {"drop_rate": 0}
net_paramsDec = {"drop_rate": 0}
inputmodule_paramsDec = {}



def AEConfigs(Config):

    if Config == '1':
        # CONFIG1
        net_paramsEnc['block_configs'] = [[32, 32], [64, 64]]
        net_paramsEnc['stride'] = [[1, 2], [1, 2]]
        net_paramsDec['block_configs'] = [[64, 32], [
            32, inputmodule_paramsEnc['num_input_channels']]]
        net_paramsDec['stride'] = net_paramsEnc['stride']
        net_paramsEnc["drop_rate"] = 0.2
        net_paramsDec["drop_rate"] = 0.2
        inputmodule_paramsDec['num_input_channels'] = net_paramsEnc['block_configs'][-1][-1]

    elif Config == '2':
        # CONFIG 2
        net_paramsEnc['block_configs'] = [[32], [64], [128], [256]]
        net_paramsEnc['stride'] = [[2], [2], [2], [2]]
        net_paramsDec['block_configs'] = [[128], [64], [32],
                                          [inputmodule_paramsEnc['num_input_channels']]]
        net_paramsDec['stride'] = net_paramsEnc['stride']
        inputmodule_paramsDec['num_input_channels'] = net_paramsEnc['block_configs'][-1][-1]

    elif Config == '3':
        # CONFIG3
        net_paramsEnc['block_configs'] = [[32], [64], [64]]
        net_paramsEnc['stride'] = [[1], [2], [2]]
        net_paramsDec['block_configs'] = [[64], [32], [
            inputmodule_paramsEnc['num_input_channels']]]
        net_paramsDec['stride'] = net_paramsEnc['stride']
        inputmodule_paramsDec['num_input_channels'] = net_paramsEnc['block_configs'][-1][-1]

    return net_paramsEnc, net_paramsDec, inputmodule_paramsDec

###################### EXPERIMENT PARAMETRES AE #########################


inputmodule_paramsEnc = {}
inputmodule_paramsEnc['num_input_channels'] = 3

########################### CREACIÓ MODEL ####################################

print("CREANT MODEL")
# CONFIG1
Config = '3'
net_paramsEnc, net_paramsDec, inputmodule_paramsDec = AEConfigs(Config)
model = AutoEncoderCNN(inputmodule_paramsEnc, net_paramsEnc,
                       inputmodule_paramsDec, net_paramsDec)

########################## CARREGUEM MODEL #################################

# Step 1: Load the pre-trained model
net_paramsEnc, net_paramsDec, inputmodule_paramsDec = AEConfigs(Config)
model = AutoEncoderCNN(inputmodule_paramsEnc, net_paramsEnc,
                       inputmodule_paramsDec, net_paramsDec)
model.load_state_dict(torch.load('model_rapid_nouCONFIG3.pth'))
model.eval()  # Set the model to evaluation mode

CREANT MODEL


AutoEncoderCNN(
  (encoder): Encoder(
    (encoder): Sequential(
      (cnnblock1): _CNNBlock(
        (cnnlayer1): _CNNLayer(
          (cnn_layer): Sequential(
            (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): LeakyReLU(negative_slope=0.01, inplace=True)
            (3): Dropout(p=0, inplace=False)
          )
        )
      )
      (cnnblock2): _CNNBlock(
        (cnnlayer1): _CNNLayer(
          (cnn_layer): Sequential(
            (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): LeakyReLU(negative_slope=0.01, inplace=True)
            (3): Dropout(p=0, inplace=False)
          )
        )
      )
      (cnnblock3): _CNNBlock(
        (cnnlayer1): _CNNLayer(
          (cnn_layer): Sequential(
           

In [7]:
import torch
import numpy as np
from sklearn.metrics import classification_report
from PIL import Image
import cv2
from torchvision import transforms

# Transformación para preprocesar las imágenes
transform = transforms.Compose([transforms.ToTensor()])

# Inicializar variables para el conjunto holdout
positivesDiagnosis = {}
errors = []
labels = []

# Calcular el umbral promedio de OTPC (Optimal Thresholds Patch Classification)
optimal_patch_threshold = np.mean([fold[0] for fold in OTPC])

# Usar el mejor threshold global de OTPD (Optimal Thresholds Patients Diagnosis)
optimal_patient_threshold = np.mean([fold[0] for fold in OTPD])

# Procesar el conjunto holdout
for input_image, patID in zip(holdoutImgs, holdoutMeta):

    # Procesar la imagen
    input_image = np.transpose(input_image, (1, 2, 0))  # Convertir a HWC
    input_image_pil = Image.fromarray((input_image * 255).astype(np.uint8))
    input_tensor = transform(input_image_pil).unsqueeze(0)

    # Pasar la imagen por el modelo
    with torch.no_grad():
        output_tensor = model(input_tensor)

    # Reconstruir la imagen de salida
    output_image = output_tensor.squeeze(0).cpu().numpy()
    output_image = np.transpose(output_image, (1, 2, 0))  # Convertir a HWC
    input_image_bgr = cv2.cvtColor(input_image, cv2.COLOR_RGB2BGR)
    output_image_bgr = cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR)

    # Calcular el error f_red
    f_red = calculate_f_red(input_image_bgr, output_image_bgr, threshold_low=-20, threshold_high=20)

    # Inicializar el diccionario de conteo por paciente si no existe
    if patID not in positivesDiagnosis:
        positivesDiagnosis[patID] = [0, 0]

    # Contar patches totales y positivos para cada paciente
    positivesDiagnosis[patID][1] += 1
    if f_red > optimal_patch_threshold:
        positivesDiagnosis[patID][0] += 1


In [8]:
print(optimal_patient_threshold*100)

16.799999999999997


In [9]:

# Evaluar pacientes en base al porcentaje de patches positivos
patientDiagnosis = []
for patID, (positives, total) in positivesDiagnosis.items():
    percentage_positive = positives / total
    print(f"Paciente {patID}: {percentage_positive*100:.2f}% de patches positivos")
    print()
    # Diagnóstico final basado en el threshold de OTPD
    if percentage_positive >= optimal_patient_threshold:
        patientDiagnosis.append([patientsRealDiagnosis[patID], 1])  # Diagnóstico positivo
        print('Diagnostico: 1')
    else:
        patientDiagnosis.append([patientsRealDiagnosis[patID], -1])  # Diagnóstico negativo
        print('Diagnostico: 0')

# Extraer etiquetas reales y predichas
true_labels = [diag[0] for diag in patientDiagnosis]
predicted_labels = [diag[1] for diag in patientDiagnosis]



Paciente B22-01: 0.00% de patches positivos

Diagnostico: 0
Paciente B22-02: 56.00% de patches positivos

Diagnostico: 1
Paciente B22-03: 1.00% de patches positivos

Diagnostico: 0
Paciente B22-04: 26.00% de patches positivos

Diagnostico: 1
Paciente B22-05: 44.00% de patches positivos

Diagnostico: 1
Paciente B22-06: 45.00% de patches positivos

Diagnostico: 1
Paciente B22-07: 0.00% de patches positivos

Diagnostico: 0
Paciente B22-08: 10.00% de patches positivos

Diagnostico: 0
Paciente B22-09: 49.00% de patches positivos

Diagnostico: 1
Paciente B22-100: 39.00% de patches positivos

Diagnostico: 1
Paciente B22-105: 5.00% de patches positivos

Diagnostico: 0
Paciente B22-10: 64.00% de patches positivos

Diagnostico: 1
Paciente B22-11: 59.00% de patches positivos

Diagnostico: 1
Paciente B22-120: 28.00% de patches positivos

Diagnostico: 1
Paciente B22-128: 50.00% de patches positivos

Diagnostico: 1
Paciente B22-12: 55.00% de patches positivos

Diagnostico: 1
Paciente B22-130: 71.00%

In [10]:
# Calculem les metriques de validació
report = classification_report(true_labels, predicted_labels, target_names=["Negative", "Positive"], output_dict=True)
PositiveRecall = report["Positive"]["recall"]
NegativeRecall = report["Negative"]["recall"]
precision = precision_score(true_labels, predicted_labels, pos_label= 1)  
recall = recall_score(true_labels, predicted_labels, pos_label= 1)
f1 = f1_score(true_labels, predicted_labels, pos_label = 1)
confusionMatrix = confusion_matrix(true_labels, predicted_labels)
    
# Imprimir resultados
print(f"Recall+: {PositiveRecall}")
print(f"Recall-: {NegativeRecall}")
print(f"Precisió (Precision): {precision}")
print(f"Sensibilitat (Recall): {recall}")
print(f"F1-score: {f1}")
print("Matriu de Confusió:")
print(confusionMatrix)
    
# Reporte detallado
print("\nReporte de clasificación:")
print(classification_report(true_labels, predicted_labels, labels=[1, -1]))

Recall+: 0.39655172413793105
Recall-: 0.7241379310344828
Precisió (Precision): 0.5897435897435898
Sensibilitat (Recall): 0.39655172413793105
F1-score: 0.4742268041237113
Matriu de Confusió:
[[42 16]
 [35 23]]

Reporte de clasificación:
              precision    recall  f1-score   support

           1       0.59      0.40      0.47        58
          -1       0.55      0.72      0.62        58

    accuracy                           0.56       116
   macro avg       0.57      0.56      0.55       116
weighted avg       0.57      0.56      0.55       116

